In [8]:
import os
import rasterio
import numpy as np
from rasterio.fill import fillnodata
import tools
import matplotlib.pyplot as plt
import geopandas as gpd
import pandas as pd

In [2]:
# INPUT/OUTPUT
dem_original_fp = '/Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/DEM/Krycklan_2015_DEM_0.5m/Krycklan_2015_DEM.tif'
cleaned_dem_fp = '/Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/DEM/Krycklan_2015_DEM_0.5m/Krycklan_2015_DEM_cleaned.tif'

In [3]:
with rasterio.open(dem_original_fp) as src:
    data1 = src.read(1)
    meta1 = src.meta
with rasterio.open(cleaned_dem_fp) as src:
    data2 = src.read(1)
    meta2 = src.meta

In [4]:
# INPUT/OUTPUT
dem_original_fp = '/Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/DEM/Krycklan_2015_DEM_0.5m/Krycklan_2015_DEM.tif'
cleaned_dem_fp = '/Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/DEM/Krycklan_2015_DEM_0.5m/Krycklan_2015_DEM_cleaned.tif'

# Load original DEM
with rasterio.open(dem_original_fp) as src:
    profile = src.profile
    transform = src.transform
    arr = src.read(1)
    nodata_val = src.nodata if src.nodata is not None else -9999

# Step 1: Mask out specific outlier regions (example boxes)
boxes = [
    ((727842.5, 7136690.9), (728065.4, 7136488.0)),
    ((736112.5, 7133787.9), (736277.6, 7133617.0))
]

for (x1, y1), (x2, y2) in boxes:
    row_start, col_start = tools.coords_to_index(transform, x1, y1)
    row_end, col_end = tools.coords_to_index(transform, x2, y2)
    row_min, row_max = sorted([row_start, row_end])
    col_min, col_max = sorted([col_start, col_end])
    arr[row_min:row_max+1, col_min:col_max+1] = nodata_val

# Step 2: Fill nodata
max_search_distance = 25/0.5*5
mask_arr = (arr != nodata_val)
arr_filled = fillnodata(arr, mask=mask_arr, max_search_distance=max_search_distance, smoothing_iterations=0)

# Step 3: Replace zero-values outside mask with nodata
arr_filled[(arr_filled == 0.) & (~mask_arr)] = nodata_val

# Update metadata
profile.update({
    'driver': 'GTiff',
    'dtype': arr_filled.dtype,
    'compress': 'lzw',
    'nodata': nodata_val
})

# Save cleaned DEM
with rasterio.open(cleaned_dem_fp, 'w', **profile) as dst:
    dst.write(arr_filled, 1)

print(f"Cleaned DEM written to: {cleaned_dem_fp}")

Cleaned DEM written to: /Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/DEM/Krycklan_2015_DEM_0.5m/Krycklan_2015_DEM_cleaned.tif


### ROADS TO SHAPEFILES

In [5]:
gpkg_file = r'/Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/TOPOGRAFI_50/kommunikation_ln24.gpkg'
ref_fp = r'/Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/SLU_FOREST_MAP_KRYCKLAN/2010/BIRCHVOL_XX_P_10.asc'
out_dir = r'/Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/TOPOGRAFI_50/shapefiles'
tools.extract_and_clip_gpkg_layers(gpkg_file=gpkg_file, ref_raster=ref_fp, out_dir=out_dir)

Processing layer: vaglinje


/Users/jpnousu/Krycklan_GIS/tools.py:169: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  clipped.to_file(out_path)
/Users/jpnousu/miniconda3/envs/wbt/lib/python3.11/site-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'objektidentitet' to 'objektiden'
  ogr_write(
/Users/jpnousu/miniconda3/envs/wbt/lib/python3.11/site-packages/pyogrio/raw.py:723: RuntimeWarning: Field skapad create as date field, though DateTime requested.
  ogr_write(
/Users/jpnousu/miniconda3/envs/wbt/lib/python3.11/site-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'objekttypnr' to 'objekttypn'
  ogr_write(
/Users/jpnousu/miniconda3/envs/wbt/lib/python3.11/site-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'avfartsvag_pafartsvag' to 'avfartsvag'
  ogr_write(
/Users/jpnousu/miniconda3/envs/wbt/lib/python3.11/site-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/la

Saved clipped shapefile to: /Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/TOPOGRAFI_50/shapefiles/vaglinje.shp
Processing layer: ovrig_vag
Saved clipped shapefile to: /Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/TOPOGRAFI_50/shapefiles/ovrig_vag.shp
Processing layer: ralstrafik
Saved clipped shapefile to: /Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/TOPOGRAFI_50/shapefiles/ralstrafik.shp


/Users/jpnousu/Krycklan_GIS/tools.py:169: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  clipped.to_file(out_path)
/Users/jpnousu/miniconda3/envs/wbt/lib/python3.11/site-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'objektidentitet' to 'objektiden'
  ogr_write(
/Users/jpnousu/miniconda3/envs/wbt/lib/python3.11/site-packages/pyogrio/raw.py:723: RuntimeWarning: Field skapad create as date field, though DateTime requested.
  ogr_write(
/Users/jpnousu/miniconda3/envs/wbt/lib/python3.11/site-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'objekttypnr' to 'objekttypn'
  ogr_write(
/Users/jpnousu/miniconda3/envs/wbt/lib/python3.11/site-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'vagutforande' to 'vagutforan'
  ogr_write(
/Users/jpnousu/miniconda3/envs/wbt/lib/python3.11/site-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered f

In [9]:
# List of shapefile paths to combine
shapefiles = [
    '/Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/TOPOGRAFI_50/shapefiles/vaglinje.shp',
    '/Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/TOPOGRAFI_50/shapefiles/ralstrafik.shp',
    '/Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/TOPOGRAFI_50/shapefiles/ovrig_vag.shp'
]

# Read and concatenate them
gdfs = [gpd.read_file(fp) for fp in shapefiles]
combined = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True), crs=gdfs[0].crs)

# Save the combined shapefile
combined.to_file('/Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/TOPOGRAFI_50/shapefiles/combined_roads.shp')

/Users/jpnousu/miniconda3/envs/wbt/lib/python3.11/site-packages/pyogrio/raw.py:723: RuntimeWarning: Field skapad create as date field, though DateTime requested.
  ogr_write(


In [10]:
# Input shapefile
input_fp = r"/Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/CATCHMENTS/Krycklan_catchments/Krycklan catchments.shp"

# Output folder
output_dir = os.path.join(os.path.dirname(input_fp), "split_catchments")
os.makedirs(output_dir, exist_ok=True)

# Read shapefile
gdf = gpd.read_file(input_fp)

# Group by the 'Huvudomr' field and write each group to a separate shapefile
for catchment_id, catchment_gdf in gdf.groupby("Huvudomr"):
    out_fp = os.path.join(output_dir, f"catchment_{int(catchment_id)}.shp")
    catchment_gdf.to_file(out_fp)
    print(f"Saved: {out_fp}")

Saved: /Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/CATCHMENTS/Krycklan_catchments/split_catchments/catchment_1.shp
Saved: /Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/CATCHMENTS/Krycklan_catchments/split_catchments/catchment_2.shp
Saved: /Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/CATCHMENTS/Krycklan_catchments/split_catchments/catchment_3.shp
Saved: /Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/CATCHMENTS/Krycklan_catchments/split_catchments/catchment_4.shp
Saved: /Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/CATCHMENTS/Krycklan_catchments/split_catchments/catchment_5.shp
Saved: /Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/CATCHMENTS/Krycklan_catchments/split_catchments/catchment_6.shp
Saved: /Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/CATCHMENTS/Krycklan_catchments/split_catchments/catchment_7.shp
Saved: